
# Digital IC Measurement Lab  
**[CD4026BE](./CD4026BE.PDF) Decade Counter with 7-Segment Display Simulation and Analog Discovery 3**

---

## Aim of the Lab

- To study the working of the CD4026BE decade counter.
- To measure and observe digital signals using Analog Discovery 3.
- To understand how the CD4026BE drives a 7-segment display.
- To verify counting, timing, and carry-out behavior.

---

## Components Required

- CD4026BE decade counter IC  
- Common cathode 7-segment display  
- Analog Discovery 3 with WaveForms software  
- Breadboard and jumper wires  
- 220 Ω resistors for segment current limiting  
- Optional: Push button (for manual clock input)

---

## IC Overview: CD4026BE

- A decade counter with built-in 7-segment decoder.
- Counts from 0 to 9 on each clock pulse.
- Drives a 7-segment display directly (no external decoder required).
- Supports cascading through Carry Out pin.

---

## Important CD4026BE Pins

| Pin | Name           | Description                        |
|-----|----------------|------------------------------------|
| 1   | Clock          | Advances count on rising edge      |
| 2   | Clock Inhibit  | Disables clock when high           |
| 3   | Reset          | Resets count to 0                  |
| 5   | Carry Out      | Goes high after count reaches 9    |
| 6   | Display Enable | Active low; enables display        |
| 7–11, 13, 14 | Segments A–G | Connect to display through resistors |
| 15  | GND            | Ground                             |
| 16  | Vcc            | +5V power supply                   |

---

## Circuit Setup

- Pin 16 (Vcc) → +5V from Analog Discovery 3
- Pin 8 (GND) → Ground
- Pin 1 (Clock) → Wavegen output (DIO or W1)
- Pin 2 (Clock Inhibit) → GND
- Pin 3 (Display Enable) → HIGH
- Pin 15 (Reset) → GND
- Pins 6-7, 9-13 → Segment inputs of display -> DIOs of AD3
- Display common cathode → GND
- Pin 5 (Carry Out) → Can be observed or left open

---

## Experiment Steps

1. **Generate a Clock Pulse**  
   - Use Wavegen on Analog Discovery 3 to produce a 1 Hz square wave.
   - Connect to Pin 1 (Clock) of CD4026.

2. **Observe Counting Behavior**  
   - Display should count from 0 to 9, updating once per second.
   - Observe the carry-out pulse at Pin 5 after reaching 9.

3. **Timing Measurements**  
   - Use Logic Analyzer to view Clock and Carry Out signals.
   - Use cursors to measure clock period and timing delays.

4. **Frequency Sweep**  
   - Gradually increase clock frequency.

5. **Optional Manual Clocking**  
   - Use a DIO/push button instead of Wavegen.
   - Observe bouncing or glitches on Logic Analyzer.
   - Discuss need for debouncing in digital systems.

---

## Observations to Record

- Circuit diagram with pin connections
- Carry-out behavior at count = 9
- Logic Analyzer screenshots of clock and carry-out waveforms
- Any glitches or limitations observed at higher speeds

---

## Key Learnings

- CD4026 can directly drive a 7-segment display from clock input.
- Clock pulses must be clean; glitches can cause incorrect counts.
- Carry-out pin allows cascading multiple counters.
- Display response is limited by frequency and visibility.
- Analog Discovery 3 is useful for waveform generation and logic analysis.

---

## Discussion Points

- Difference between synchronous and asynchronous counters
- Role of signal integrity in digital circuits
- Use cases of decade counters in real systems (e.g., digital clocks, counters)
- Why proper segment current limiting is necessary

---
       ┌───── a ─────┐
       │             │
     f │             │ b
       │             │
       ├───── g ─────┤
       │             │
     e │             │ c
       │             │
       └───── d ─────┘


In [49]:
from ctypes import *              # import the C compatible data types
import time
from sys import platform, path    # this is needed to check the OS type and get the PATH
from os import sep                 # OS specific file path separators
import matplotlib.pyplot as plt
import numpy
import datetime
import os
import sys
import tkinter as tk
%gui tk

In [50]:
if sys.platform.startswith("win"):
    dwf = cdll.dwf
elif sys.platform.startswith("darwin"):
    dwf = cdll.LoadLibrary("/Library/Frameworks/dwf.framework/dwf")
else:
    dwf = cdll.LoadLibrary("libdwf.so")

constants_path = "C:" + sep + "Program Files (x86)" + sep + "Digilent" + sep + "WaveFormsSDK" + sep + "samples" + sep + "py"
path.append(constants_path)
import dwfconstants as constants

In [53]:
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value) # flags to show usb devices only
# filter_flags = c_int(filter_flags.value | constants.enumfilterDemo.value) # flags to include demo devices ; comment this line to exclude
# Enumerate devices
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))
print(f"Number of devices found: {device_count.value}")
# Retrieve device name (optional)
name_buf = create_string_buffer(64) # buffer to store device name
sn_buf = create_string_buffer(32) # buffer to store device serial nmber
opn_status = c_int() # to store device opening status
status_lookup = lambda code: {
    0: "Not opened",
    1: "Opened by current process",
    2: "Opened by another process"
}.get(code, "Unknown status")

for i in range(device_count.value):
    dwf.FDwfEnumDeviceName(c_int(i), name_buf)
    dwf.FDwfEnumSN(c_int(i), sn_buf)
    dwf.FDwfEnumDeviceIsOpened(c_int(i),byref(opn_status))
    print(f"Detected device {i}\nname: {name_buf.value.decode()}\nserial: {sn_buf.value.decode()}\n{status_lookup(opn_status.value)}\n")
hdwf = c_int()
idx=int(input())
dwf.FDwfEnumDeviceIsOpened(c_int(idx),byref(opn_status))
if(opn_status.value==0):
    dwf.FDwfDeviceOpen(c_int(idx), byref(hdwf))
    if hdwf.value == 0:
        raise RuntimeError("Failed to open device.")
    print(f"device opened successfully!")
else:
    print(status_lookup(opn_status.value))

Number of devices found: 1
Detected device 0
name: Analog Discovery 3
serial: SN:210415BC9A47
Not opened

device opened successfully!


In [52]:
dwf.FDwfAnalogInReset(hdwf)
dwf.FDwfDeviceCloseAll()
dwf.FDwfDeviceClose(hdwf)

0

In [57]:
dwRead=c_uint32()
# the device will only be configured when FDwf###Configure is called
dwf.FDwfDeviceAutoConfigureSet(hdwf, c_int(1))

# enable output/mask on 8 LSB IO pins, from DIO 0 to 7
dwf.FDwfDigitalIOOutputEnableSet(hdwf, c_int(0xFF00))
# set value on enabled IO pins

# fetch digital IO information from the device 
pulse(15)
dwf.FDwfDigitalIOStatus(hdwf)
# read state of all pins, regardless of output enable
dwf.FDwfDigitalIOInputStatus(hdwf, byref(dwRead))

for i in range(16):
    print(f"DIO-{i}:", (dwRead.value>>i)&1)
# print("DIO-1:", (dwRead.value>>1)&1)

#print(dwRead as bitfield (32 digits, removing 0b at the front)
print("Digital IO Pins: ", bin(dwRead.value)[2:].zfill(16))
display=SevenSegmentDisplay(scale=3)
code=(str(bin(dwRead.value))[-7:])[::-1]
print(code)
if(len(code)==7):
    disp_update()
else:
    print("code error")

DIO-0: 1
DIO-1: 0
DIO-2: 1
DIO-3: 1
DIO-4: 1
DIO-5: 1
DIO-6: 1
DIO-7: 0
DIO-8: 0
DIO-9: 0
DIO-10: 0
DIO-11: 0
DIO-12: 0
DIO-13: 0
DIO-14: 0
DIO-15: 1
Digital IO Pins:  1000000001111101
1011111


In [55]:
def pulse(pin):
    dwf.FDwfDigitalIOOutputSet(hdwf,c_int(0))
    dwf.FDwfDigitalIOConfigure(hdwf)
    time.sleep(.1)
    dwf.FDwfDigitalIOOutputSet(hdwf,c_int(1<<pin))
    dwf.FDwfDigitalIOConfigure(hdwf)
    time.sleep(.1)
    dwf.FDwfDigitalIOStatus(hdwf)
    dwf.FDwfDigitalIOInputStatus(hdwf, byref(dwRead))
    code=(str(bin(dwRead.value))[-7:])[::-1]
    # dwf.FDwfDigitalIOOutputSet(hdwf,c_int(0))
    # dwf.FDwfDigitalIOConfigure(hdwf)
    # time.sleep(.1)

def get_code():
    return (str(bin(dwRead.value))[-7:])[::-1]

def disp_update():
    code=get_code()
    display.update_display(code)
    display.root.after(100,disp_update)

In [54]:
SEG_ORDER = ['a', 'b', 'c', 'd', 'e', 'f', 'g']

SEGMENTS_COORDS = {
    'a': [(15, 5), (65, 5), (55, 15), (25, 15)],
    'b': [(65, 5), (75, 15), (75, 65), (65, 55)],
    'c': [(65, 75), (75, 85), (75, 135), (65, 125)],
    'd': [(15, 135), (65, 135), (55, 125), (25, 125)],
    'e': [(5, 75), (15, 85), (15, 135), (5, 125)],
    'f': [(5, 5), (15, 15), (15, 65), (5, 55)],
    'g': [(15, 70), (65, 70), (55, 80), (25, 80)],
}

class SevenSegmentDisplay:
    def __init__(self, scale=3):
        self.scale = scale
        self.root = tk.Tk()
        self.root.title("7-Segment Display")
        self.canvas_w = 80 * scale
        self.canvas_h = 140 * scale
        self.canvas = tk.Canvas(self.root, width=self.canvas_w, height=self.canvas_h, bg='black')
        self.canvas.pack()
        self.segment_items = {}
        self.button = tk.Button(self.root, text="Next Digit", command=self.button_callback)
        self.button.pack(pady=10)
        # Draw segments initially (all off)
        for seg_name in SEG_ORDER:
            poly = self._draw_segment(SEGMENTS_COORDS[seg_name], False)
            self.segment_items[seg_name] = poly

    def _draw_segment(self, coords, on):
        scaled_coords = []
        for x, y in coords:
            scaled_coords.append(x * self.scale)
            scaled_coords.append(y * self.scale)
        color = 'red' if on else 'gray30'
        return self.canvas.create_polygon(scaled_coords, fill=color, outline='black', width=1)

    def update_display(self, code):
        if len(code) != 7 or any(c not in '01' for c in code):
            raise ValueError("Input must be a 7-bit binary string (e.g. '1010101').")

        for i, bit in enumerate(code):
            seg_name = SEG_ORDER[i]
            color = 'red' if bit == '1' else 'gray30'
            self.canvas.itemconfig(self.segment_items[seg_name], fill=color)
        self.canvas.update()
        self.root.update_idletasks() 
        # self.root.update()

    def run(self):
        self.root.update()
    
    def button_callback(self):
        pulse(15)
        disp_update()

    

# Usage example:

if __name__ == "__main__":
    display = SevenSegmentDisplay(scale=3)
    
    # Example: cycle through some numbers after the window is up
    import time

    # Binary codes for digits 0-9 on 7-seg a-g (common cathode)
    digits = [
        "1111110", # 0
        "0110000", # 1
        "1101101", # 2
        "1111001", # 3
        "0110011", # 4
        "1011011", # 5
        "1011111", # 6
        "1110000", # 7
        "1111111", # 8
        "1111011"  # 9
    ]

    def update_loop():
        for code in digits:
            display.update_display(code)
            # display.root.update()
            time.sleep(1)
        display.root.after(1000, update_loop)  # loop indefinitely

    display.root.after(100, update_loop)
    display.run()